### 1. LightFM 구현

In [2]:
# 필요한 라이브러리 불러오기
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
import time

#### 1-1. Dataset

In [3]:
# ✅ 1. Movielens 데이터셋 불러오기 (평점 4점 이상만 사용)
data = fetch_movielens(indicator_features=False, genre_features=True)
# LightFM이 사용하는 MovieLens 100k 원본 파일 경로
url = "http://files.grouplens.org/datasets/movielens/ml-100k/u.item"
# 해당 파일에는 | 구분자, 인코딩은 ISO-8859-1 사용
movies = pd.read_csv(url, sep='|', header=None, encoding='latin-1')
# 영화 제목만 추출
item_labels = movies[1].values
# 원본 데이터 대입
data['item_labels'] = item_labels

#### 1-2. Model

In [4]:
# ✅ 2. LightFM 모델 학습 (WARP Loss 은 활용하지 못해서 logistic 활용)
model = LightFM(loss='logistic')

In [5]:
try:
    start = time.time()
    model.fit(data['train'], epochs=10, num_threads=1)
    print("✅ 3단계: 모델 학습 완료!", f"({time.time() - start:.2f}초)")
except Exception as e:
    print("❌ model.fit 중 오류 발생:", e)

✅ 3단계: 모델 학습 완료! (1.97초)


#### 1-3. 함수 정의 및 출력

In [6]:
# ✅ 3. 추천 함수 정의
def recommend_movies(model, user_id, num_items=5):
    n_items = data['train'].shape[1]
    scores = model.predict(user_id, np.arange(n_items))
    top_items = np.argsort(-scores)[:num_items]  # 높은 점수순으로 정렬
    return [data['item_labels'][i] for i in top_items]

# ✅ 4. 특정 사용자에게 추천 실행
user_id = 3
recommended_movies = recommend_movies(model, user_id)

print(f"\n✅ 사용자 {user_id}에게 추천하는 영화:")
for i, movie in enumerate(recommended_movies, start=1):
    print(f"{i}. {movie}")


✅ 사용자 3에게 추천하는 영화:
1. Star Wars (1977)
2. English Patient, The (1996)
3. Contact (1997)
4. Liar Liar (1997)
5. Fargo (1996)


### 2. Hybrid 모델로 확장

#### 2-1. 유저 메타데이터 로드 및 전처리

In [7]:
# 유저 정보 로딩
user_info_url = "http://files.grouplens.org/datasets/movielens/ml-100k/u.user"
users = pd.read_csv(user_info_url, sep='|', header=None, encoding='latin-1')
users.columns = ['user_id', 'age', 'gender', 'occupation', 'zip_code']

# user_id를 0부터 시작하게 맞춤
users['user_id'] = users['user_id'] -1
users = users.sort_values('user_id').reset_index(drop=True)

# 직업과 성별을 범주형으로 인코딩 (예: "gender=F", "occupation=student")
users['gender'] ='gender=' + users['gender']
users['occupation'] ='occupation=' + users['occupation']

# 나이대 범주형 데이터도 추가로 인코딩
def age_group(age):
    if age < 20:
        return 'age=10s'
    elif age < 30:
        return 'age=20s'
    elif age < 40:
        return 'age=30s'
    elif age < 50:
        return 'age=40s'
    else:
        return 'age=50+'

users['age'] = users['age'].apply(age_group)

# 특성 컬럼만 선택
user_features_raw = users[['gender','occupation','age']]

#### 2-2. LightFM용 user_features 만들기(CSR 포맷)

In [8]:
from lightfm.data import Dataset

# 새 데이터셋 생성
dataset = Dataset()

# 사용자와 아이템 수 정의
num_users, num_items = data['train'].shape
dataset.fit(users=range(num_users), items=range(num_items))

# 유저 특성 등록
all_user_features = list(np.unique(user_features_raw.values.flatten()))
dataset.fit_partial(user_features=all_user_features)

# 사용자별 특성 맵핑
user_features_tuple = [
    (user_id, list(user_features_raw.iloc[user_id].values)) for user_id in range(num_users)
]

# CSR 행렬 생성
user_features = dataset.build_user_features(user_features_tuple)

#### 2-3. 모델에 유저 특성 추가해서 학습

In [10]:
# 모델 새로 정의
model_hybrid = LightFM(loss ='logistic')

# hybrid 모델 학습
model_hybrid.fit(data['train'],
                 user_features=user_features,
                 item_features=data['item_features'],
                 epochs=10,
                 num_threads=1)

#### 2-4. 추천 함수 수정 (user_features, item_features 포함)

In [11]:
def recommend_hybrid(model, user_id, num_items=5):
    n_items = data['train'].shape[1]
    scores = model.predict(user_id, np.arange(n_items),
                           user_features=user_features,
                           item_features=data['item_features'])
    top_items = np.argsort(-scores)[:num_items]
    return [data['item_labels'][i] for i in top_items]

#### 2-5. 추천 결과 보기

In [12]:
user_id = 3
recommended = recommend_hybrid(model_hybrid, user_id)

print(f" Hybrid 모델 - 사용자 {user_id}에게 추천하는 영화:")
for i, movie in enumerate(recommended, 1):
    print(f"{i}. {movie}")

 Hybrid 모델 - 사용자 3에게 추천하는 영화:
1. Empire Strikes Back, The (1980)
2. From Dusk Till Dawn (1996)
3. Army of Darkness (1993)
4. Muppet Treasure Island (1996)
5. Diva (1981)


### 3. 모델 평가하기

In [13]:
from lightfm.evaluation import precision_at_k, auc_score

# Collaborative 모델 평가 (기본 모델)
precision = precision_at_k(model, data['test'], k=5).mean()
auc = auc_score(model, data['test']).mean()

print(f"Precision@5 : {precision:.4f}")
print(f"AUC Score : {auc:.4f}")

Precision@5 : 0.0889
AUC Score : 0.8436


In [14]:
# Hybrid 모델 평가 (user/item features 사용한 모델)
precision_h = precision_at_k(model_hybrid,
                             data['test'],
                             user_features=user_features,
                             item_features=data['item_features'],
                             k=5).mean()
auc_h = auc_score(model_hybrid,
                data['test'],
                user_features=user_features,
                item_features=data['item_features']).mean()

print(f"Precision@5 : {precision_h:.4f}")
print(f"AUC Score : {auc_h:.4f}")

Precision@5 : 0.0123
AUC Score : 0.5992


### 4. LightFM 모델 저장하기

In [22]:
import pickle
import os
save_dir = 'models'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    print(f"폴더 {save_dir} 생성 완료")
else:
    pass

with open('models/lightfm_model.pkl', "wb") as f:
    pickle.dump(model_hybrid, f)
    
print(" 모델 저장 완료: lightfm_model.pkl")

폴더 models 생성 완료
 모델 저장 완료: lightfm_model.pkl


In [23]:
# 함께 저장하면 좋은 것들
## (선택) 영화 제목 리스트 및 하이브리드 모델의 경우 필요한 유저, 아이템 정보

with open(f"{save_dir}/item_labels.pkl", "wb") as f:
    pickle.dump(data['item_labels'], f)
print("item_labels 저장 완료!")

with open(f"{save_dir}/user_features.pkl", "wb") as f:
    pickle.dump(user_features, f)
print("user_features 저장 완료!")

with open(f"{save_dir}/item_features.pkl", "wb") as f:
    pickle.dump(data['item_features'], f)
print("item_features 저장 완료!")

item_labels 저장 완료!
user_features 저장 완료!
item_features 저장 완료!


In [24]:
# 저장이 잘 되었는지 확인
with open(f"{save_dir}/lightfm_model.pkl", 'rb') as f:
    loaded_model = pickle.load(f)

print('모델 로딩 성공!')

모델 로딩 성공!
